In [1]:
!pip install transformers trl==0.18.1 datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import shutil
import pandas as pd
import torch
from accelerate import PartialState
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig
)
from trl import (
    PPOConfig,
    PPOTrainer,
    ModelConfig,
    ScriptArguments,
    get_peft_config,
)
from trl.trainer.utils import SIMPLE_CHAT_TEMPLATE


# ====== PATHS (adjust these) ======
SFT_MODEL_PATH = "/content/drive/MyDrive/tinyllama-lora-sft-tuned-model"
REWARD_MODEL_PATH = "EleutherAI/pythia-1b-deduped"
CSV_PATH = "/content/drive/MyDrive/PPO_prompt_dataset.csv"
OUTPUT_DIR = "/content/drive/MyDrive/ppo_output"


# ====== Custom Dataset Class ======
class CustomArgs:
    dataset_name = None
    dataset_config = None
    dataset_train_split = "train"
    dataset_test_split = None


In [11]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 13.0 MB/s eta 0:00:00


In [6]:
from transformers import PreTrainedTokenizerBase


In [ ]:
if __name__ == "__main__":
    script_args = CustomArgs()
    training_args = PPOConfig(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=1,            
        gradient_accumulation_steps=8,
        learning_rate=3e-6,
        total_episodes=5000,
        eval_strategy="no",
        response_length=32,                       
        stop_token="eos",
        seed=42
    )
    model_config = ModelConfig(
        model_name_or_path=SFT_MODEL_PATH,
        trust_remote_code=True,
        torch_dtype="auto",
        use_peft=True,
    )

    shutil.rmtree(training_args.output_dir, ignore_errors=True)

    tokenizer = AutoTokenizer.from_pretrained(
        model_config.model_name_or_path,
        padding_side="left"
    )
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    if tokenizer.chat_template is None:
        tokenizer.chat_template = SIMPLE_CHAT_TEMPLATE

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
    )

    policy_model = AutoModelForCausalLM.from_pretrained(
        model_config.model_name_or_path,
        device_map="auto",
        quantization_config=bnb_config,
        trust_remote_code=True
    )
    policy_model.gradient_checkpointing_enable()

    reward_model = AutoModelForSequenceClassification.from_pretrained(
        REWARD_MODEL_PATH,
        num_labels=1,
        device_map="auto",
        quantization_config=bnb_config,
        trust_remote_code=True
    )
    value_model = AutoModelForSequenceClassification.from_pretrained(
        REWARD_MODEL_PATH,
        num_labels=1,
        device_map="auto",
        quantization_config=bnb_config,
        trust_remote_code=True
    )

    ref_policy = None
    peft_config = get_peft_config(model_config)

    df = pd.read_csv(CSV_PATH)

    assert "prompt" in df.columns, " Missing 'prompt' column in CSV."

    dataset = Dataset.from_pandas(df)

    def format_dataset(example):
        return {
            "messages": [{"role": "user", "content": example["prompt"]}]
        }

    dataset = dataset.map(format_dataset)
    print(dataset)

    def tokenize(example):
      prompt_text = tokenizer.apply_chat_template(
          example["messages"],
          add_generation_prompt=True,
          return_tensors=None,
          tokenize=False,
      )

      tokenized = tokenizer(prompt_text, truncation=True, max_length=512, return_tensors="pt")
      return {
          "input_ids": tokenized["input_ids"][0],
          "attention_mask": tokenized["attention_mask"][0],
          "lengths": tokenized["input_ids"].shape[-1]
      }

    with PartialState().local_main_process_first():
        dataset = dataset.map(tokenize, remove_columns=dataset.column_names)
        dataset = dataset.filter(lambda x: x["lengths"] <= 512)
        print("Samples after filtering:", len(dataset))

        dataset.set_format(type="torch")
    train_test_split = dataset.train_test_split(test_size=0.1, seed=42) # Adjust test_size as needed
    train_dataset = train_test_split["train"]
    eval_dataset = train_test_split["test"]


    trainer = PPOTrainer(
        args=training_args,
        model=policy_model,
        ref_model=ref_policy,
        processing_class=tokenizer,
        reward_model=reward_model,
        value_model=value_model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        peft_config=peft_config,
    )



Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at EleutherAI/pythia-1b-deduped and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPTNeoXForSequenceClassification were not initialized from the model checkpoint at EleutherAI/pythia-1b-deduped and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/6236 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'messages'],
    num_rows: 6236
})


Map:   0%|          | 0/6236 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6236 [00:00<?, ? examples/s]

✅ Samples after filtering: 6236


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
    trainer.train()
#https://github.com/huggingface/trl/blob/main/examples/scripts/ppo/ppo_tldr.py

===training policy===


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ query                                          ┃ model response                                ┃ score          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ <|user|>                                       │ Tesla GPUs offer several benefits for         │ 0.736328125    │
│ How can cloud-based applications benefit from  │ cloud-based applications, including:          │                │
│ Tesla GPUs?                                    │                                               │                │
│ <|assistant|>                                  │ 1. High performance: Tesla GPUs are designed  │                │
│                                                │ to deliver                                    │                │
├────────────────────────────────────────────────┼───────────────────────────────────────────────┼────────────────┤
│ <|user|>                                       │ The cudaOccupancyMaxPotentialBlockSize API    │ -1.37890625    │
│ How does the                                   │ simplifies launch configuration by providing  │                │
│ cudaOccupancyMaxPotentialBlockSize API         │ a single function that takes a block size as  │                │
│ simplify launch configuration?                 │ an argument. This                             │                │
│ <|assistant|>                                  │                                               │                │
│                                                │                                               │                │
├────────────────────────────────────────────────┼───────────────────────────────────────────────┼────────────────┤
│ <|user|>                                       │ There are several resources available for     │ 0.8974609375   │
│ Where can readers find additional information  │ readers interested in learning more about the │                │
│ about the implementation and use of the        │ implementation and use of the register cache  │                │
│ register cache?                                │ in the Java Virtual Machine (JVM).            │                │
│ <|assistant|>                                  │                                               │                │
│                                                │ 1                                             │                │
├────────────────────────────────────────────────┼───────────────────────────────────────────────┼────────────────┤
│ <|user|>                                       │ The purpose of cudaStreamSynchronize(stream)  │ 0.485595703125 │
│ What is the purpose of                         │ in CUDA is to wait for the GPU to finish      │                │
│ cudaStreamSynchronize(stream) in CUDA?         │ executing the command that was executed on    │                │
│ <|assistant|>                                  │ the                                           │                │
│                                                │                                               │                │
├────────────────────────────────────────────────┼───────────────────────────────────────────────┼────────────────┤
│ <|user|>                                       │ The CUDA sample code deviceQuery provides     │ 1.318359375    │
│ What information can be obtained from the CUDA │ information about the device that the program │                │
│ sample code deviceQuery?                       │ is running on, including its name, vendor,    │                │
│ <|assistant|>                                  │ and model. This information can               │                │
│                                                │                                               │                │
└────────────────────────────────────────────────┴──────

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
